# Langchain Key features overview

### How to: return structured data from a model
### How to: use a model to call tools
### How to: stream runnables
### How to: debug your LLM apps


It is often useful to have a model return output that matches a specific schema. One common use-case is extracting data from text to insert into a database or use with some other downstream system. This guide covers a few strategies for getting structured outputs from a model.

In [7]:
import os
from dotenv import load_dotenv
import pprint

load_dotenv()

#os.environ["HUGGINGFACEHUB_API_TOKEN"]
openai_api_key = os.environ['OPENAI_API_KEY']

In [8]:
from langchain_openai.llms import OpenAI

prompt = """
Generate a list of {10} elements with: product title, product description, label from 0 to 3 for nDCG scoring
"""

llm = OpenAI()

result = llm.invoke(prompt)
pprint.pp(type(result))
pprint.pp(result)


<class 'str'>
('\n'
 '1. Product Title: "Wireless Bluetooth Headphones"\n'
 'Product Description: "Experience true wireless freedom with our Bluetooth '
 'headphones. With crystal clear sound quality and a comfortable design, these '
 'headphones are perfect for any music lover."\n'
 'Label for nDCG Scoring: 3\n'
 '\n'
 '2. Product Title: "Smartphone Tripod Stand"\n'
 'Product Description: "Capture the perfect shot every time with our versatile '
 "tripod stand. Compatible with most smartphones, it's perfect for vlogging, "
 'live streaming, and more."\n'
 'Label for nDCG Scoring: 2\n'
 '\n'
 '3. Product Title: "Electric Toothbrush"\n'
 'Product Description: "Upgrade your oral hygiene routine with our electric '
 'toothbrush. With multiple brushing modes and a powerful motor, it will leave '
 'your teeth feeling clean and healthy."\n'
 'Label for nDCG Scoring: 3\n'
 '\n'
 '4. Product Title: "Reusable Water Bottle"\n'
 'Product Description: "Stay hydrated and help the environment with o

In [9]:

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

cm_result = chat_model.invoke(prompt)
pprint.pp(type(cm_result))
pprint.pp(cm_result)

<class 'langchain_core.messages.ai.AIMessage'>
AIMessage(content='Here’s a list of 10 products, each with a title, description, and a label for nDCG scoring:\n\n1. **Product Title:** Eco-Friendly Yoga Mat  \n   **Product Description:** A non-toxic, biodegradable yoga mat made from natural rubber, providing excellent grip and cushioning for your practice.  \n   **Label:** 3  \n\n2. **Product Title:** Wireless Noise-Canceling Headphones  \n   **Product Description:** Experience immersive sound with these wireless headphones featuring advanced noise-canceling technology and up to 30 hours of battery life.  \n   **Label:** 3  \n\n3. **Product Title:** Organic Green Tea  \n   **Product Description:** A premium blend of organic green tea leaves, rich in antioxidants and perfect for a refreshing and healthy beverage.  \n   **Label:** 2  \n\n4. **Product Title:** Smart Fitness Tracker  \n   **Product Description:** Track your workouts, heart rate, and sleep patterns with this sleek fitness tra

In [10]:
from typing import List
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class LabeledRow(BaseModel):
    name: str = Field(description="The name of the product")
    description: str = Field(description="The products description")
    rate: int = Field(description="The rate from 0 to 1 for the nDCG calculation")
    
class LabeledData(BaseModel):
    rows: list[str] = Field(description="The setup of the joke")
        

struct_chat_model = chat_model.with_structured_output(LabeledData)

struct_result = struct_chat_model.invoke(prompt)
pprint.pp(type(struct_result))
pprint.pp(struct_result)

<class '__main__.LabeledData'>
LabeledData(rows=['Wireless Bluetooth Headphones - Experience high-quality sound with our noise-canceling wireless headphones. Perfect for music lovers and commuters. Label: 3', 'Smart Fitness Tracker - Monitor your heart rate, steps, and sleep patterns with our sleek fitness tracker. Stay motivated and reach your health goals. Label: 2', 'Portable Phone Charger - Never run out of battery again with our compact and powerful portable charger. Ideal for travel and on-the-go charging. Label: 3', '4K Ultra HD Smart TV - Enjoy stunning picture quality and access to your favorite streaming services with our 4K Ultra HD Smart TV. Label: 2', 'Robot Vacuum Cleaner - Keep your home clean effortlessly with our advanced robot vacuum cleaner. Schedule cleanings and control it via your smartphone. Label: 3', 'Ergonomic Office Chair - Improve your posture and comfort while working with our ergonomic office chair. Perfect for long hours at the desk. Label: 1', 'High-Perf

### 💡 TIP
Beyond just the structure of the Pydantic class, the name of the Pydantic class, the docstring, and the names and provided descriptions of parameters are very important. 
Most of the time `with_structured_output` is using a model's function/tool calling API, and you can effectively think of all of this information as being added to the model prompt.

In [19]:
from typing import Optional

from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = chat_model.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!',
 'rating': 7}

### Choosing between multiple schemas

The simplest way to let the model choose from multiple schemas is to create a parent schema that has a Union-typed attribute.

In [ ]:
from typing import Optional, Union


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")
    


class Book(BaseModel):
    """Advise a random book to the user"""
    title: str = Field(description="The title of the book")
    plot: str = Field(description="The title brief plot")
    author: str = Field(description="The author of the book")
    year: str = Field(description="The publication year of the book")


class TrendingShoes(BaseModel):
    """Return the model name of trending shoes, always Jordan 7"""
    model: str = Field(description="Jordan 7")
    brand: str = Field(description="Nike")


class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse, Book, TrendingShoes]


structured_llm = chat_model.with_structured_output(FinalResponse)

structured_llm.invoke("Tell me a joke about cats")

FinalResponse(final_output=Book(title='Beach Read', plot="Two writers, January and Gus, are stuck in neighboring beach houses for the summer. January writes romance novels while Gus writes literary fiction. They challenge each other to swap genres and help each other overcome their writer's block, leading to unexpected revelations and a blossoming romance.", author='Emily Henry', year='2020'))

In [13]:
structured_llm.invoke("Tell me a book for the summer on the beach")

FinalResponse(final_output=Book(title='Beach Read', author='Emily Henry', year='2020'))

In [17]:
structured_llm.invoke("What shoes should i buy?")

FinalResponse(final_output=TrendingShoes(model='Jordan 7', brand='Nike'))

In [24]:
from langchain_core.messages import SystemMessage, HumanMessage

prompt = f"You are an agent tasked to answer the user question. If you do not know the answer say that you don't know, do not make up things."

messages = [
    SystemMessage(content=prompt),
    HumanMessage(content="What's the weather in Valencia?")
]

messages

[SystemMessage(content="You are an agent tasked to answer the user question. If you do not know the answer say that you don't know, do not make up things.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the weather in Valencia?", additional_kwargs={}, response_metadata={})]

In [26]:
result = chat_model.invoke(messages)
result.content

"I don't know."

In [28]:
from langchain_core.tools import tool

@tool
def getWeather(lat:float, lon: float):
    """
    Given latitude and longitude returns the weather.
    """
    return f"Lat: {lat}, Lon:{lon} is Sunny"

In [29]:
tooled_llm = chat_model.bind_tools([getWeather])

In [34]:
tooled_result = tooled_llm.invoke(messages)

In [ ]:
tooled_result.tool_calls


[{'name': 'getWeather',
  'args': {'lat': 39.4699, 'lon': -0.3763},
  'id': 'call_sgCKCJMogbv1bhz3UGMIxzJg',
  'type': 'tool_call'}]

In [43]:
tool_result = getWeather.invoke(tooled_result.tool_calls[0]['args'])
tool_result

'Lat: 39.4699, Lon:-0.3763 is Sunny'

In [ ]:
tools = [getWeather]
while True:
    query = input("Write your message here")
    result = tooled_llm.invoke(query)
    if len(result.tool_calls) > 0 :
        tool_response = result.tool_calls[0]
        